In [17]:
from Scripts import preprocessing as prep
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix

In [23]:
# amazon_link = '../Data/amazon_phone.pkl'
amazon_link = '../Data/amazon_movie.pkl'
df = pd.read_pickle(amazon_link)

In [24]:
df.text = prep.preprocess_reviews(df.text)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.label, test_size=0.25,random_state=7, stratify=df.label)

In [26]:
print('Training target statistics: {}'.format(Counter(y_train), sorted(y_train)))
print('Testing target statistics: {}'.format(Counter(y_test), sorted(y_test)))

Training target statistics: Counter({5.0: 677719, 4.0: 286691, 3.0: 150697, 1.0: 77993, 2.0: 76680})
Testing target statistics: Counter({5.0: 225907, 4.0: 95564, 3.0: 50232, 1.0: 25998, 2.0: 25560})


In [31]:
vectorizer = TfidfVectorizer(min_df=5,max_df=0.9, ngram_range=(1,3))
# vectorizer = CountVectorizer(min_df=5,max_df=0.9, ngram_range=(1,3))
train_vectorized = vectorizer.fit_transform(X_train)
print ("Feature size of TF-IDF: ", len(vectorizer.get_feature_names()))
test_vectorized = vectorizer.transform(X_test)

MemoryError: 

In [ ]:
vectorizer_fs = SelectKBest(score_func=f_classif, k=1000)
fs_train_vectorized = vectorizer_fs.fit_transform(train_vectorized,y_train)
fs_test_vectorized = vectorizer_fs.transform(test_vectorized)

In [ ]:
clf = LogisticRegression(C=1.0, dual=False, fit_intercept=True, random_state=0, solver='lbfgs', intercept_scaling=1, max_iter=100, multi_class='multinomial', class_weight='balanced')
clf.fit(fs_train_vectorized, y_train)
y_pred = clf.predict(fs_test_vectorized)

In [ ]:
accuracy = str(metrics.accuracy_score(y_test, y_pred))
precision = str(metrics.precision_score(y_test, y_pred, average="macro"))
f1 = str(metrics.f1_score(y_test, y_pred, average="macro"))
print("Accuracy:" + accuracy)
print("Precision:" + precision)
print("F1:" + f1)
print(pd.crosstab(y_test, y_pred))

# Visualization of Confusion Matrix and saving
plt.rcParams['figure.facecolor'] = 'white'
title = f"Confusion matrix 75_25)"
disp = plot_confusion_matrix(clf, fs_test_vectorized, y_test,
                             display_labels=[1.0, 2.0, 3.0, 4.0, 5.0],
                             cmap=plt.cm.Blues)
disp.ax_.set_title(title)
plt.show()

title_norm = title + "_normalize"
disp_norm = plot_confusion_matrix(clf, fs_test_vectorized, y_test,
                             display_labels=[1.0, 2.0, 3.0, 4.0, 5.0],
                             cmap=plt.cm.Blues,
                             normalize='true')
disp_norm.ax_.set_title(title_norm)
plt.show()

In [ ]:
# For Movie (Countvectorizer)
# Accuracy:0.5884855916325861
# Precision:0.46344580886772385
# F1:0.48193706273773207


# For Phone (Complete)
# Accuracy:0.5967368855844692
# Precision:0.45642294744267653
# F1:0.4737592372355196